Итак, возвращаемся к нашему другу Коле и его данным по букмекерской конторе.  

Коля хочет расширять бизнес и увеличивать прибыль, и мы поможем ему 
с помощью аналитических навыков и знания нужных инструментов. 
Работать будем с теми же данными - log.csv и users.csv.

Что мы уже сделали?

Загрузили данные (и побороли ошибки, связанные с плохим форматом данных).
Дали признакам (колонкам) соответствующие имена.
Научились фильтровать данные и применять к ним apply.
Обнаружили часть ошибок в данных.

Какие вопросы остались?
Вспомним часть вопросов из прошедшего модуля.

Сколько раз человеку надо прийти, чтобы сделать ставку?

Каков средний выигрыш в процентах?

Каков баланс по каждому пользователю?

Какие города самые выгодные?

В каких городах самая высокая ставка?

Сколько в среднем времени проходит от первого посещения сайта до первой попытки?

Мы постараемся ответить не только на эти, но и на многие другие вопросы.

Какие бывают пропуски
Вот список значений, которые по умолчанию считаются как пропуски: '', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null'.  

In [1]:
import pandas as pd
import numpy as np

In [2]:
log = pd.read_csv('log.xls')
log

,Запись пользователя № - user_919,[2019-01-01 14:06:51,Unnamed: 2,Unnamed: 3
0,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
1,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
2,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
4,Запись пользователя № - user_917,[2019-01-01 23:34:55,156789.0,NaN
...,...,...,...,...
994,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,NaN
995,#error,NaN,10054.0,29265.0
996,#error,NaN,10454.0,NaN
997,#error,NaN,1000.0,NaN


В pandas есть метод isna(), который возвращает таблицу такой же размерности, что и на вход, но значения в ней - True или False. True, если данное значение является пропуском, и False в ином случае.

In [3]:
log.head(5).isna()

,Запись пользователя № - user_919,[2019-01-01 14:06:51,Unnamed: 2,Unnamed: 3
0,False,False,True,True
1,False,False,True,True
2,False,False,True,True
3,False,False,True,True
4,False,False,False,True


In [4]:
log.columns = ['user','time','bet','win']
log.head(5)

,user,time,bet,win
0,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
1,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
2,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
4,Запись пользователя № - user_917,[2019-01-01 23:34:55,156789.0,NaN


Посчитайте количество пропусков в столбце time. Метод isna() есть не только у DataFrame, но и у Series. Это значит, что применять его можно не только ко всей таблице, но и к каждому столбцу отдельно.

In [5]:
log['time'].isna().value_counts()

False    984
True      15
Name: time, dtype: int64

Удаление пропусков
Пропуски можно удалять автоматически. Во многих случаях это правильно, так как данные с большим количеством пропусков часто не имеют смысла и не приносят никакой пользы.  

Удалять данные с пропусками можно с помощью метода dropna().

Параметр axis в методе dropna() говорит методу, по какой оси удалять значения.  

1. Если нужно удалить строки, в которых встречается пропуск (NaN), следует указать axis=0.  Зачем это делать? Например, у нас из 1000 примеров данных про пользователей пропуски есть в пяти. Разумно их удалить, так как их количество пренебрежимо мало.

2. Если нужно удалить столбцы, в которых встречается пропуск (NaN), нужно указывать axis=1. Зачем? Иногда в одном конкретном столбце пропусков настолько много, что с ними просто не хочется возиться - смысла в них все равно почти нет. 

Еще один интересный параметр - subset. Что он делает? Если передать в него список значений по одной оси (например, названия столбцов) и задать при этом в параметре axis другую ось (в нашем случае 0), то мы удалим те строки, для которых в данных столбцах находится пропуск. То же самое работает и наоборот: нужно поменять axis на 1 и вместо названий столбцов передавать индексы строк.

In [6]:
log.head(5)
log.dropna(axis = 1)

,user
0,Запись пользователя № - user_973
1,Запись пользователя № - user_903
2,Запись пользователя № - user_954
3,Запись пользователя № - user_954
4,Запись пользователя № - user_917
...,...
994,Запись пользователя № - user_984
995,#error
996,#error
997,#error


In [7]:
log.dropna(axis = 0)

,user,time,bet,win
13,Запись пользователя № - user_917,[2019-01-02 8:57:36,145732.0,1987653.0
28,Запись пользователя № - user_942,[2019-01-04 13:59:42,1678321.0,9876543.0
150,Запись пользователя № - user_982,[2019-01-16 21:54:22,100.0,4749.0
188,Запись пользователя № - user_964,[2019-01-21 18:34:44,200.0,4667.0
204,Запись пользователя № - user_931,[2019-01-22 5:26:59,300.0,4319.0
...,...,...,...,...
966,Запись пользователя № - user_975,[2019-04-19 22:25:15,1000.0,6108.0
970,Запись пользователя № - user_912,[2019-04-20 10:35:49,10554.0,31799.0
971,Запись пользователя № - user_926,[2019-04-20 10:35:50,10354.0,30244.0
975,Запись пользователя № - user_970,[2019-04-20 10:35:54,10354.0,30691.0


In [10]:
log.head(5)

,user,time,bet,win
0,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
1,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
2,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
4,Запись пользователя № - user_917,[2019-01-01 23:34:55,156789.0,NaN


In [11]:
log.isna()

,user,time,bet,win
0,False,False,True,True
1,False,False,True,True
2,False,False,True,True
3,False,False,True,True
4,False,False,False,True
...,...,...,...,...
994,False,False,False,True
995,False,True,False,False
996,False,True,False,True
997,False,True,False,True


In [13]:
log['user'].isna().value_counts()

False    999
Name: user, dtype: int64

In [14]:
log['time'].isna().value_counts()

False    984
True      15
Name: time, dtype: int64

In [16]:
del log['time']

In [17]:
log

,user,bet,win
0,Запись пользователя № - user_973,NaN,NaN
1,Запись пользователя № - user_903,NaN,NaN
2,Запись пользователя № - user_954,NaN,NaN
3,Запись пользователя № - user_954,NaN,NaN
4,Запись пользователя № - user_917,156789.0,NaN
...,...,...,...
994,Запись пользователя № - user_984,9754.0,NaN
995,#error,10054.0,29265.0
996,#error,10454.0,NaN
997,#error,1000.0,NaN


Дубли  - это повторяющиеся строки в данных. В сложных случаях строки могут быть практически одинаковые, но не совсем.

Самая частая причина очень банальна: дубли появляются из-за человеческих ошибок или невнимательности.

Например, при добавлении записи в систему вы случайно два раза нажали на кнопку "добавить". Если система позволяет иметь одинаковые записи, поздравляю - у вас в данных появились дубли.

Еще одна причина - слияние баз данных. Например, вы переносите телефоны из контактной книжки (физической, в которую заносили номера ручкой на бумагу, такие были популярны в прошлом) в телефон. Назвали в одном месте Сашу Сашей, а в другом Александром. Да, это тоже дубль, просто сам случай немного более сложный.

Сложным случаям типа последнего можно посвятить очень много времени, так что это материал для другого, более продвинутого курса по анализу данных. Скажу лишь, что в каждом конкретном случае подозрительные случаи нужно рассматривать отдельно, это требует кропотливой работы. 

Мы же рассмотрим простой случай, когда у вас в данных есть идентичные строки.

Как удалить простые дубли
В pandas есть метод для удаления дублей (дубликатов) - drop_duplicates(). Он просто удаляет повторяющиеся строки:

01. import pandas as pd  
02. df = pd.read_csv('data.csv')
03. df.drop_duplicates()  
У данного метода тоже есть параметр subset, в этом случае нужно передавать список содержащий названия столбцов.  

In [18]:
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    999 non-null    object 
 1   bet     485 non-null    float64
 2   win     138 non-null    float64
dtypes: float64(2), object(1)
memory usage: 23.5+ KB


In [96]:
log = pd.read_csv('log.xls')
log.columns = ['user','time', 'bet', 'win']


In [97]:
log_ut = log[['user','time']]

In [98]:
log_ut.drop_duplicates()

,user,time
0,Запись пользователя № - user_973,[2019-01-01 14:51:16
1,Запись пользователя № - user_903,[2019-01-01 16:31:16
2,Запись пользователя № - user_954,[2019-01-01 17:17:51
3,Запись пользователя № - user_954,[2019-01-01 21:31:18
4,Запись пользователя № - user_917,[2019-01-01 23:34:55
...,...,...
990,Запись пользователя № - user_965,[2019-04-20 12:55:41
991,Запись пользователя № - user_967,[2019-04-20 14:59:36
992,Запись пользователя № - user_973,[2019-04-20 17:09:56
993,Запись пользователя № - user_977,[2019-04-20 18:10:07


In [99]:
log_ut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   user    999 non-null    object
 1   time    984 non-null    object
dtypes: object(2)
memory usage: 15.7+ KB


In [100]:
log_time = log['time'].dropna()

In [101]:
log.time = log.time.apply(lambda x:str(x).replace('[','') if ('[' in str(x)) else x)


In [102]:
log = log.dropna()

In [103]:
import datetime
from datetime import datetime,timedelta

In [104]:
time_list = []
for i in log.time:
    i = str(i)
    time_list.append(datetime.strptime(i, '%Y-%m-%d %H:%M:%S'))
max(time_list)



datetime.datetime(2019, 4, 20, 12, 55, 41)

In [88]:
max(log.time)

'2019-04-20 12:55:41'

In [108]:
log = pd.read_csv("log.xls")  
log = log.dropna()  
log.columns = ['user_id', 'time', 'bet', 'win']  
log['time'] = log['time'].apply(lambda x: x[1:])  
log['time'] = pd.to_datetime(log['time'])  
# log['time'] = log.time.apply(lambda x: x.minute)
log['time'].head() 

13    2019-01-02 08:57:36
28    2019-01-04 13:59:42
150   2019-01-16 21:54:22
188   2019-01-21 18:34:44
204   2019-01-22 05:26:59
Name: time, dtype: datetime64[ns]

In [111]:
log['year'] = log['time'].dt.year

In [112]:
log

,user_id,time,bet,win,year
13,Запись пользователя № - user_917,2019-01-02 08:57:36,145732.0,1987653.0,2019
28,Запись пользователя № - user_942,2019-01-04 13:59:42,1678321.0,9876543.0,2019
150,Запись пользователя № - user_982,2019-01-16 21:54:22,100.0,4749.0,2019
188,Запись пользователя № - user_964,2019-01-21 18:34:44,200.0,4667.0,2019
204,Запись пользователя № - user_931,2019-01-22 05:26:59,300.0,4319.0,2019
...,...,...,...,...,...
966,Запись пользователя № - user_975,2019-04-19 22:25:15,1000.0,6108.0,2019
970,Запись пользователя № - user_912,2019-04-20 10:35:49,10554.0,31799.0,2019
971,Запись пользователя № - user_926,2019-04-20 10:35:50,10354.0,30244.0,2019
975,Запись пользователя № - user_970,2019-04-20 10:35:54,10354.0,30691.0,2019
